In [ ]:
# Run for the first time and restart session
!pip install accelerate -U -q
!pip install transformers[torch] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.1 MB/s eta 0:00:00


Connect Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Requirements

In [ ]:
%%capture
!pip install kaggle
!pip install jiwer
!pip install datasets
!pip install transformers
!pip install wandb
!pip install pyloudnorm

Check GPU info


In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.cuda.get_device_name(0)  # Get the name of the GPU device
    print("Running on GPU:", device)
else:
    print("Running on CPU")

Running on GPU: Tesla V100-SXM2-16GB


In [ ]:
!nvidia-smi

Sun Mar 10 14:24:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              25W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Prepare to download dataset from kaggle
Upload kaggle.json containing username and API key in your drive


In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Download Dataset from kaggle

In [ ]:
!kaggle datasets download anishshilpakar/cleaned-asr-data

100% 4.12G/4.12G [02:27<00:00, 33.9MB/s]
100% 4.12G/4.12G [02:28<00:00, 29.9MB/s]


In [ ]:
# !kaggle datasets download aashish12321/openslr-large-nepali-dataset-cleaned

100% 3.77G/3.77G [02:19<00:00, 32.0MB/s]
100% 3.77G/3.77G [02:19<00:00, 29.1MB/s]


Unzip Dataset

In [ ]:
%%capture
!unzip cleaned-asr-data.zip

In [ ]:
# %%capture
# !unzip openslr-large-nepali-dataset-cleaned.zip

Imports


In [ ]:
from datasets import Audio, ClassLabel, load_dataset, load_metric
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from torch._C import device
from transformers import (
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    trainer_utils
)

In [ ]:
import json
import os
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import IPython.display as ipd
import jiwer
import librosa
import numpy as np
import pandas as pd
import random
import re
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
import torchaudio
import wandb
import soundfile as sf
import pyloudnorm as pyln

Connect to WANDB to store logs


In [ ]:
wandb.login(key="197e5bd9893a682ed805464a97513fec783153cf")
wandb.init(project='wav2vec2-nepali-v1-itr-4-mod')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: william21111340. Use `wandb login --relogin` to force relogin


Connect to huggingface to save models

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
%%capture
!apt install git-lfs

In [ ]:
project_folder_location = '/content/drive/MyDrive/wav2vec2-nepali-v1-itr-4-mod'

In [ ]:
%cd {project_folder_location}

/content/drive/MyDrive/wav2vec2-nepali-v1-itr-4-mod


In [ ]:
!mkdir csv_files
!mkdir model_files
!mkdir checkpoints
!mkdir tokenizer
!mkdir processor_files

In [ ]:
csv_folder_path=project_folder_location+'/csv_files'
trained_model_path=project_folder_location+'/model_files'
processor_path=project_folder_location+'/processor_files'
checkpoints_path=project_folder_location+'/checkpoints'
tokenizer_path=project_folder_location+'/tokenizer'

In [ ]:
colnames = ["path","labels"]
audio_path = '/content/data/data/audio/'
# audio_path = '/content/drive/MyDrive/major_project/asr/data/audio/'
transcript_path = '/content/transcript_durations/dataset_duration_4sec.csv'
df = pd.read_csv(transcript_path,usecols=colnames)
df["path"] = audio_path + df["path"] + ".flac"
df=df[0:20000] # Taking 20K data
print(df.shape)
df.head()

(20000, 2)


,path,labels
0,/content/data/data/audio/nep_3154_4861507228.flac,निर्स टोस्ट बेकरी एन्ड रेस्टुरेन्टले छोटो समयम...
1,/content/data/data/audio/nep_3154_4933069146.flac,ऐतिहासिक महत्त्व बोकेको यो दरबार भक्तपुर सहरको...
2,/content/data/data/audio/nep_6587_6577731096.flac,इन्टरप्राइज प्रडक्टसन पार्टनर्स एल पी ले यसका ...
3,/content/data/data/audio/4adc138892.flac,यसको साथै यो सहर व्यापार र वाणिज्य केन्द्रका र...
4,/content/data/data/audio/nep_6587_8996660673.flac,न्यु योर्क लाइफ इन्स्योरेन्स कम्पनीका कृष्ण पो...


In [ ]:
# TEST_RATIO = 0.90
VAL_RATIO = 0.8

In [ ]:
# train_df, test_df = train_test_split(df, random_state = 0, train_size = TEST_RATIO)
train_df, val_df = train_test_split(df, random_state = 0, train_size = VAL_RATIO)

In [ ]:
train_df.to_csv(csv_folder_path+'/train.csv',index=False)
val_df.to_csv(csv_folder_path+'/val.csv',index=False)
# test_df.to_csv(csv_folder_path+'/test.csv',index=False)
!ls {csv_folder_path}

train.csv  val.csv


In [ ]:
data_files = {
    "train": "train.csv",
    "validation": "val.csv",
    # "test": "test.csv"
}

In [ ]:
train_data = load_dataset(csv_folder_path, data_files = data_files, split = "train")
val_data = load_dataset(csv_folder_path, data_files = data_files, split = "validation")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_data.remove_columns(["path"]), num_examples = 5)

,labels
0,यो नेपाली सेनाको पनि हतियार चिनो हो।
1,लासको थुप्रोसँगै इस्मा
2,विश्वका सबै शासकहरूले किमको व्यक्तित्वको प्रशंसा गरेका छन्
3,प्रस्तावहरूमा कुनै सकारात्मक
4,गरिनुका साथै प्रसिद्ध


In [ ]:
chars_to_ignore = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["labels"] = re.sub(chars_to_ignore, '', batch["labels"]).lower() + " "
    return batch

In [ ]:
train_data = train_data.map(remove_special_characters)
val_data = val_data.map(remove_special_characters)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
vocab_path = '/content/data/data/vocabulary/vocab.json'
# vocab_path = '/content/drive/MyDrive/major_project/asr/data/vocabulary/vocab.json'
tokenizer = Wav2Vec2CTCTokenizer(
    vocab_path,
    unk_token = "[UNK]",
    pad_token = "[PAD]",
    word_delimiter_token = "|"
)

In [ ]:
repo_name="wav2vec2-nepali-v1-itr-4-mod"

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/nadika/wav2vec2-nepali-v1-itr-4-mod/commit/bfdeb7974bf2852b1a2db6db5e5c8d1d7895d908', commit_message='Upload tokenizer', commit_description='', oid='bfdeb7974bf2852b1a2db6db5e5c8d1d7895d908', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.save_pretrained(tokenizer_path)

('/content/drive/MyDrive/wav2vec2-nepali-v1-itr-4-mod/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/wav2vec2-nepali-v1-itr-4-mod/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/wav2vec2-nepali-v1-itr-4-mod/tokenizer/vocab.json',
 '/content/drive/MyDrive/wav2vec2-nepali-v1-itr-4-mod/tokenizer/added_tokens.json')

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size = 1,
    sampling_rate = 16000,
    padding_value = 0.0,
    do_normalize = True,
    return_attention_mask = True
)

In [ ]:
processor = Wav2Vec2Processor(
    feature_extractor = feature_extractor,
    tokenizer = tokenizer
)

In [ ]:
processor.push_to_hub(repo_name)

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nadika/wav2vec2-nepali-v1-itr-4-mod/commit/b44262dcf1191ef625955ce92ee833846a2def32', commit_message='Upload processor', commit_description='', oid='b44262dcf1191ef625955ce92ee833846a2def32', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
processor.save_pretrained(processor_path)

[]

In [ ]:
train_data

Dataset({
    features: ['path', 'labels'],
    num_rows: 16000
})

In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["labels"]

    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000

    return batch

In [ ]:
train_data = train_data.map(speech_file_to_array_fn, remove_columns=train_data.column_names)
val_data = val_data.map(speech_file_to_array_fn, remove_columns=val_data.column_names)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
rand_int = random.randint(0, len(train_data) - 1)
ipd.Audio(data = np.asarray(
    train_data[rand_int]["speech"]),
    autoplay = True,
    rate = 16000
)

In [ ]:
rand_int = random.randint(0, len(train_data) - 1)

print("Target text:", train_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", train_data[rand_int]["sampling_rate"])

Target text: राष्ट्रिय स्तरमा दर्ताभएका 
Input array shape: (70400,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch, processor):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate = batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
train_data = train_data.map(prepare_dataset, fn_kwargs = {"processor": processor}, remove_columns = train_data.column_names, batch_size = 16, num_proc = 4, batched = True)
val_data = val_data.map(prepare_dataset, fn_kwargs = {"processor": processor}, remove_columns = val_data.column_names, batch_size = 16, num_proc = 4, batched = True)

Map (num_proc=4):   0%|          | 0/16000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor = processor, padding = True)

In [ ]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer", revision = "master")

<ipython-input-63-ba2b58d813dc>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for cer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/master/metrics/cer/cer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    # cer_best = 1 - cer because load best model is considering greater value of cer for better results
    cer_best = 1 - cer
    # wandb.log({'wer':wer, 'cer':cer})
    # experiment.log_metric('wer', wer)
    # experiment.log_metric('cer', cer)

    return {"wer": wer, "cer": cer, "accuracy": cer_best}

In [ ]:
prev_model_path='Ashwini1412/wav2vec2-nepali-v1-itr-3'

In [ ]:
# model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-large-xlsr-53",
#     attention_dropout = 0.1,
#     hidden_dropout = 0.1,
#     feat_proj_dropout = 0.0,
#     mask_time_prob = 0.05,
#     layerdrop = 0.1,
#     gradient_checkpointing = True,
#     ctc_loss_reduction = "mean",
#     pad_token_id = processor.tokenizer.pad_token_id,
#     vocab_size = len(processor.tokenizer)
# )
# # model.config.ctc_zero_infinity = True

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    prev_model_path,
    attention_dropout = 0.12,
    hidden_dropout = 0.12,
    feat_proj_dropout = 0.0,
    mask_time_prob = 0.05,
    layerdrop = 0.1,
#     gradient_checkpointing = True,
    ctc_loss_reduction = "mean",
    pad_token_id = processor.tokenizer.pad_token_id,
    vocab_size = len(processor.tokenizer)
).to("cuda")

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
# model = Wav2Vec2ForCTC.from_pretrained(prev_model_path)

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
model.freeze_feature_encoder()

In [ ]:
hub_model='nadika/wav2vec2-nepali-v1-itr-4-mod' # Username + repo_name where our new model is to be pushed

In [ ]:
training_args = TrainingArguments(
  output_dir = repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=15,
  fp16 = True,
  save_steps = 200,
  eval_steps=200,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=50,
  save_total_limit = 2,
  overwrite_output_dir=True,
  gradient_checkpointing = True,
  report_to = "wandb",
  metric_for_best_model = 'accuracy',
  load_best_model_at_end=True,
  push_to_hub=True,
  hub_model_id=hub_model
)

In [ ]:
trainer = Trainer(
    model = model,
    data_collator = data_collator,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = processor.feature_extractor,
)

20K data for 15 epochs

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer,Cer,Accuracy
200,0.333900,0.137809,0.517324,0.123452,0.876548
400,0.334700,0.143388,0.521533,0.123900,0.876100
600,0.295900,0.145542,0.523036,0.125332,0.874668
800,0.289700,0.149584,0.524540,0.123920,0.876080
1000,0.294800,0.146834,0.530177,0.129031,0.870969
1200,0.281300,0.147489,0.525442,0.127510,0.872490
1400,0.272100,0.142180,0.519805,0.126973,0.873027
1600,0.248500,0.148142,0.524239,0.126316,0.873684
1800,0.261400,0.148592,0.521007,0.123651,0.876349
2000,0.256500,0.146505,0.522210,0.125889,0.874111


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran